In [ ]:
!wget https://perso.esiee.fr/~gueurett/LV_Research/dataset_audio.zip
!unzip dataset_audio.zip

In [ ]:
# if wget desn't work (happens sometimes I believe it's because my school servers are blocking the access), import the dataset manualy and unzip on this block
!unzip dataset_audio.zip

In [ ]:
!pip install pydub

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from pydub import AudioSegment
from sklearn import preprocessing
from scipy.io.wavfile import read

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

---
---

# **Preprocessing**



In [ ]:
def delete_excess(data):
  return data[:length_audio]

def add_average(data):
  result = np.zeros(shape=(length_audio,))
  end = data.shape[0]
  mean = np.mean(data[end-25:])
  result[:end] = data
  result[end:] = mean
  return result

In [ ]:
directory = "./dataset_audio/"
file_name = "I0474.wav"
path = directory + file_name

print(path)

rate, data = read(path)

print("rate = ", rate)
print("data.shape = ", data.shape)

length = data.shape[0] / rate
print("length(s): ", length, "s")
length_audio = data.shape[0]
print("length of unique audio:", length_audio)
nb_channels = data.shape[1]
print("number of channels: ", nb_channels)

n_samples = 0
for file_name in os.listdir(directory):
  n_samples += 1
print("number of files:", n_samples)

dataset = np.zeros(shape=(n_samples, length_audio))

# creating the dataset and labels

labels = np.zeros(shape=(n_samples,))

# reshaping the data if not exactly 4s
for index, filename in enumerate(os.listdir(directory)):
  if filename[0] == 'I': labels[index] = 1    # indoor is 1, outdoor is 0
  path = directory + filename
  rate, data = read(path)
  data = np.mean(data,axis=1)
  dim = data.shape
  if dim != (length_audio, nb_channels):
    if dim[0] > length_audio:
      result = delete_excess(data)
    else:
      result = add_average(data)
    dataset[index] = result
  else:
    dataset[index] = data

dataset = (dataset - np.mean(dataset)) / np.std(dataset)  # let's normalize

print("----------------------------------")
print("| dataset.shape: ", dataset.shape)
print("| labels.shape : ", labels.shape)
print("----------------------------------")

./dataset_audio/I0474.wav
rate =  44100
data.shape =  (176400, 2)
length(s):  4.0 s
length of unique audio: 176400
number of channels:  2
number of files: 200
----------------------------------
| dataset.shape:  (200, 176400)
| labels.shape :  (200,)
----------------------------------


In [ ]:
spliter = 0.2
index = int(n_samples * (1-spliter))

train_x = dataset[:index]
test_x  = dataset[index:]

train_y = labels[:index]
test_y  = labels[index:]

print("----------------------------------")
print("| train_x.shape: ", train_x.shape)
print("| train_y.shape : ", train_y.shape)
print("----------------------------------")
print("| test_x.shape: ", test_x.shape)
print("| test_y.shape : ", test_y.shape)
print("----------------------------------")

----------------------------------
| train_x.shape:  (160, 176400)
| train_y.shape :  (160,)
----------------------------------
| test_x.shape:  (40, 176400)
| test_y.shape :  (40,)
----------------------------------


---
---

# **AdaBOOST**

In [ ]:
ada = AdaBoostClassifier(n_estimators=100, random_state=8)
ada.fit(train_x, train_y)

print(ada.score(test_x, test_y))

0.625


---
---
# **KNearestNeighbors**

In [ ]:
kneigh = KNeighborsClassifier(n_neighbors=150, leaf_size=50)
kneigh.fit(train_x, train_y)

print(kneigh.score(test_x, test_y))

0.675


---
---

# **Decision Tree Classifier**

In [ ]:
tree = DecisionTreeClassifier(max_depth=1000, min_samples_split=5, min_samples_leaf=5, max_features='auto', random_state=8)
tree.fit(train_x, train_y)

print(tree.score(test_x, test_y))

0.625


---
---
# **Naive Bayes**

In [ ]:
nb = GaussianNB()
nb.fit(train_x, train_y)

print(nb.score(test_x, test_y)) # 0.35 == 0.65 ??

0.35


---
---
# **SVM**

In [ ]:
svm = SVC()
svm.fit(train_x, train_y)

print(svm.score(test_x, test_y))

0.675


---
---
# **Logistic Regression**

In [ ]:
lr = LogisticRegression(random_state=8)
lr.fit(train_x, train_y)

print(lr.score(test_x, test_y))

0.525


---
---
# **RandomForestClassifier**

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', oob_score=True, random_state=8)
rfc.fit(dataset, labels)

print(rfc.oob_score_)

0.67


---
---
# **XGBoost**

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(dataset, labels, test_size=0.3, random_state=8)

model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [ ]:
model.fit(train_x, train_y)

XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)

In [ ]:
predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions)
print(accuracy)

0.6833333333333333


---
---
# **Linear Regression**

In [ ]:
linreg = LinearRegression()

linreg.fit(train_x, train_y)
print(linreg.score(test_x, test_y))

-55.25416384136285
